In [9]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)

In [10]:
input_files = [ 
                'decIP_src_ipaddr_cache.csv',
                'decIP_dst_ipaddr_cache.csv',
                'src_port_cache.csv',
                'dst_port_cache.csv',
                'protocol_cache.csv',
                'flow_ack_counter.csv',
                'flow_psh_counter.csv',
                'flow_syn_counter.csv',
                'flow_active_start_time_stamp.csv',
                'flow_duration.csv',
                'flow_total_inactive_duration.csv',
                'flow_total_packets.csv',
                'forward_total_packets.csv',
                # 'ipaddr_cache.csv',
                'flow_active_segments.csv',
                'flow_current_time_stamp.csv',
                'flow_min_duration.csv',
                'flow_start_time_stamp.csv',
                'flow_total_active_duration.csv',
                'flow_total_length.csv',
                'forward_total_length.csv',
                'fw_win_byt.csv'
                #'src_ipaddr_cache.csv',
                #'dst_ipaddr_cache.csv',
              ]

In [11]:
X = pd.DataFrame()
for file in input_files:
    column_name = os.path.splitext(file)[0]
    x = pd.read_csv("../" + file, header=None).T
    #x.columns = [column_name]
    X = pd.concat((X, x.squeeze().rename(column_name)), axis=1) #, ignore_index=True)
    
# delete last row as it is empty one that came form p4 registry reading
X = X[:-1]

# trim spaces
X = X.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# trim unmapped hashes in register
X = X[X['decIP_dst_ipaddr_cache'] != "..."]

# Packet lenght mean
X['Packet Length Mean'] = X['flow_total_length'] / X['flow_total_packets']
X['Subflow Fwd Bytes'] = X['forward_total_packets']

In [12]:
print("Number of flows:", len(X))
print("total packets:", sum(X['flow_total_packets']))

Number of flows: 112
total packets: 1356.0


In [13]:
X.columns

Index(['decIP_src_ipaddr_cache', 'decIP_dst_ipaddr_cache', 'src_port_cache',
       'dst_port_cache', 'protocol_cache', 'flow_ack_counter',
       'flow_psh_counter', 'flow_syn_counter', 'flow_active_start_time_stamp',
       'flow_duration', 'flow_total_inactive_duration', 'flow_total_packets',
       'forward_total_packets', 'flow_active_segments',
       'flow_current_time_stamp', 'flow_min_duration', 'flow_start_time_stamp',
       'flow_total_active_duration', 'flow_total_length',
       'forward_total_length', 'fw_win_byt', 'Packet Length Mean',
       'Subflow Fwd Bytes'],
      dtype='object')

In [14]:
X

,decIP_src_ipaddr_cache,decIP_dst_ipaddr_cache,src_port_cache,dst_port_cache,protocol_cache,flow_ack_counter,flow_psh_counter,flow_syn_counter,flow_active_start_time_stamp,flow_duration,flow_total_inactive_duration,flow_total_packets,forward_total_packets,flow_active_segments,flow_current_time_stamp,flow_min_duration,flow_start_time_stamp,flow_total_active_duration,flow_total_length,forward_total_length,fw_win_byt,Packet Length Mean,Subflow Fwd Bytes
19,192.168.10.05,192.168.10.03,49189.0,88.0,6.0,8.0,2.0,2.0,36809273.0,1592.0,0.0,10.0,10.0,0.0,36810865.0,0.0,36809273.0,0.0,3524.0,3524.0,8192.0,352.400000,10.0
24,192.168.10.05,192.168.10.03,49193.0,389.0,6.0,14.0,8.0,2.0,37136888.0,574299.0,0.0,16.0,16.0,0.0,37711187.0,0.0,37136888.0,0.0,4740.0,4740.0,8192.0,296.250000,16.0
137,192.168.10.05,192.168.10.03,49188.0,49666.0,6.0,40.0,28.0,2.0,71086479.0,34277917.0,71086479.0,42.0,42.0,1.0,71086647.0,0.0,36808730.0,3205605.0,13798.0,13798.0,8192.0,328.523810,42.0
279,192.168.10.05,23.15.04.09,49229.0,80.0,6.0,4.0,1.0,1.0,167631719.0,71963422.0,167631719.0,5.0,5.0,1.0,167631719.0,0.0,95668297.0,93327.0,396.0,396.0,8192.0,79.200000,5.0
491,72.21.91.29,192.168.10.05,80.0,49208.0,6.0,5.0,2.0,1.0,88309501.0,4787794.0,0.0,5.0,5.0,0.0,93097295.0,0.0,88309501.0,0.0,1804.0,1804.0,65535.0,360.800000,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7721,192.168.10.05,172.217.10.131,49214.0,443.0,6.0,72.0,12.0,1.0,90229095.0,2880049.0,0.0,73.0,73.0,0.0,93109144.0,0.0,90229095.0,0.0,5170.0,5170.0,8192.0,70.821918,73.0
7947,192.168.10.05,23.15.04.19,49198.0,80.0,6.0,6.0,1.0,1.0,148319629.0,109772021.0,148319629.0,7.0,7.0,1.0,148334902.0,0.0,38562881.0,47425.0,492.0,492.0,65535.0,70.285714,7.0
7957,192.168.10.05,23.15.04.19,49230.0,80.0,6.0,5.0,1.0,1.0,167631514.0,71963145.0,167631514.0,6.0,6.0,1.0,167631514.0,0.0,95668369.0,33456.0,468.0,468.0,8192.0,78.000000,6.0
8100,131.253.34.240,192.168.10.05,443.0,49234.0,6.0,7.0,3.0,1.0,109634804.0,290057.0,0.0,7.0,7.0,0.0,109924861.0,0.0,109634804.0,0.0,5102.0,5102.0,8192.0,728.857143,7.0


In [15]:
# save p4 metrics data
X.to_csv('../p4_metrics.csv', index=False)

In [ ]:
# read CICFlowmeter metrics and trim off unused columns
B = pd.read_csv('../tcp_test.pcap_Flow.csv')
A = B[['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp',  'Subflow Fwd Bytes', 'Flow Duration', 'Flow IAT Min', 'PSH Flag Count', 'SYN Flag Count', 'Packet Length Mean', 'Total Length of Fwd Packet', 'Active Mean', 'Active Min', 'ACK Flag Count', 'FWD Init Win Bytes', 'Label']]
A.to_csv('../tcp_test.pcap_Flow_trimmed.csv', index=False)